In [25]:
# mount drive https://datascience.stackexchange.com/questions/29480/uploading-images-folder-from-my-system-into-google-colab
# login with your google account and type authorization code to mount on your googlbie drive.
from google.colab import drive
drive.mount('/gdrive')
root = '/gdrive/My Drive/CS492I/project'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [13]:
root = '.'

## 1. Collect CodeSearchNet Repositories

In [ ]:
import json

import pandas as pd
from pathlib import Path
pd.set_option('max_colwidth',300)
from pprint import pprint

In [ ]:
!wget https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
!mkdir CodeSearchNet
!unzip python.zip -d CodeSearchNet

--2021-12-03 07:32:59--  https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.69.238
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.69.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 940909997 (897M) [application/zip]
Saving to: ‘python.zip’

python.zip          100%[===================>] 897.32M  65.4MB/s    in 13s     

2021-12-03 07:33:12 (68.1 MB/s) - ‘python.zip’ saved [940909997/940909997]

Archive:  python.zip
   creating: CodeSearchNet/python/
   creating: CodeSearchNet/python/final/
   creating: CodeSearchNet/python/final/jsonl/
   creating: CodeSearchNet/python/final/jsonl/train/
  inflating: CodeSearchNet/python/final/jsonl/train/python_train_9.jsonl.gz  
  inflating: CodeSearchNet/python/final/jsonl/train/python_train_12.jsonl.gz  
  inflating: CodeSearchNet/python/final/jsonl/train/python_train_10.jsonl.gz  
  inflating: CodeSearchNet/python/final/jsonl/train/pyt

In [ ]:
python_files = sorted(Path('CodeSearchNet/python').glob('**/*.gz'))
print(python_files)

[PosixPath('CodeSearchNet/python/final/jsonl/test/python_test_0.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_0.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_1.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_10.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_11.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_12.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_13.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_2.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_3.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_4.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_5.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_6.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_7.jsonl.gz

In [ ]:
columns_long_list = ['repo', 'path', 'url', 'code', 
                     'code_tokens', 'docstring', 'docstring_tokens', 
                     'language', 'partition']

def jsonl_list_to_dataframe(file_list, columns=columns_long_list):
    """Load a list of jsonl.gz files into a pandas DataFrame."""
    return pd.concat([pd.read_json(f, 
                                   orient='records', 
                                   compression='gzip',
                                   lines=True)[columns] 
                      for f in file_list], sort=False)

In [ ]:
pydf = jsonl_list_to_dataframe(python_files, columns=['repo'])
pydf = pydf.drop_duplicates().reset_index(drop=True)
pydf.to_pickle(f"{root}/repos.pkl")

## 2. Collect diff and commits

In [15]:
!pip install pydriller
!pip install pandas
!pip install nltk

In [14]:
from pydriller import *
import pandas as pd
from nltk.tokenize import regexp_tokenize
import re
from itertools import chain
import json

In [15]:
repodf = pd.read_pickle(f"{root}/repos.pkl")
print(repodf.shape)
print(repodf)

(13590, 1)
                                 repo
0                     soimort/you-get
1                      apache/airflow
2                      pytorch/vision
3                      asciimoo/searx
4              tensorflow/probability
...                               ...
13585         praekelt/python-ambient
13586                 zenreach/py-era
13587  TakesxiSximada/custom_settings
13588            openpermissions/bass
13589               xnuinside/clifier

[13590 rows x 1 columns]


In [16]:
def basic_filter(message):
    return message.split("\n", 1)[0].strip()

# Remove [label] in front of commit if exists
def label_filter(message):
    if (message.startswith('[')):
        end_bracket_index = message.find(']')
        if (end_bracket_index == -1):
            return None
        return message[end_bracket_index+1:]
    return message

def camel_case_split(str):
    return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)

def case_splitter(token):
    return list(map(lambda x: x.lower(), camel_case_split(token[0].upper() + token[1:]) if token[0].isalpha() else [token]))


def split_by_quote(diff):
    text_list = [(0,0)]
    quote_state = 0 #["out","single","double","Single","Double","quote"]
    i=0
    while i<len(diff):
      if diff[i] == "\\":i+=1
      elif (diff[i] == "\n" and quote_state == 5) or (diff[i] == '#' and quote_state == 0):
        text_list.append((quote_state,i))
        quote_state = abs(5-quote_state)
      elif diff[i] == '"':
        if i+2<len(diff) and diff[i:i+3]=='"""' and (quote_state == 0 or quote_state ==4):
          text_list.append((quote_state,i+max(quote_state-1,0)))
          quote_state = abs(4-quote_state)
          i+=2
        elif quote_state == 0 or quote_state == 2:
          text_list.append((quote_state,i+max(quote_state-1,0)))
          quote_state = abs(2-quote_state)
      elif diff[i] == "'":
        if i+2<len(diff) and diff[i:i+3] == "'''" and (quote_state == 0 or quote_state == 3):
          text_list.append((quote_state,i+quote_state))
          quote_state = abs(3-quote_state)
          i+=2
        elif quote_state == 0 or quote_state == 1:
          text_list.append((quote_state,i+quote_state))
          quote_state = abs(1-quote_state)
      i+=1
    text_list.append((quote_state,i))
    return [(text_list[i][0]==0,diff[text_list[i-1][1]:text_list[i][1]]) for i in range(1,len(text_list))]
  

token_regex = r"""(?x)
<(?:add|del)>   #Filtered eariler
|(?:[-+*/^&~|=%!<>@?$][\s]*)+     #Sequence of symbols
|[\n]                    #Change row
|[a-zA-Z]+               #General text
|[0-9]+                  #Number
"""
string_regex = r"""(?x)
<(?:add|del)>
|[\n]
|[a-zA-Z]+
"""
def diff_tokenizer(diff_text):
    diff = '\n'.join(map(lambda x: x[1], filter(lambda y: y[0] % 2 == 0, enumerate(diff_text.split("@@")))))
    diff = diff.replace('\n+', '\n<add>').replace('\n-', '\n<del>')
    diff = re.sub(r"(?:\n[ \t\r\f\v]*)+","\n",diff) # Join continuous row change
    quote_split = split_by_quote(diff)
    token_initial = chain.from_iterable(map(lambda a: regexp_tokenize(a[1],(token_regex if a[0] else string_regex)), quote_split))
    token_camel_case_split = chain.from_iterable(map(case_splitter, token_initial))
    return token_camel_case_split


def commit_msg_tokenizer(msg):
    line = basic_filter(msg)
    line = label_filter(line)
    if (line is None):
        return None
    if line.count("`")%2 == 1:
        return None
    
    backtick_regex = r"`[^`]+`|[^`]+"
    line_divided_by_backtick = re.findall(backtick_regex,line)
    tokens = chain.from_iterable(map(lambda text: regexp_tokenize(text,(token_regex if text[0] == '`' else string_regex)),line_divided_by_backtick))
    commit_tokens = list(chain.from_iterable(map(case_splitter, tokens)))
    if (len(commit_tokens) < 3 or len(commit_tokens) > 30):
        return None
    if ("version" in commit_tokens):
        return None
    return commit_tokens

def parse_repo_commits(repo_name, commit_limit=50):
    data = []
    commit_count = 0
    commit_count_traversed = 0
    try:
      for commit in Repository(
          f"https://github.com/{repo_name}",
          only_modifications_with_file_types=[".py"],
          only_no_merge=True,
          order='reverse'
      ).traverse_commits():
          commit_count_traversed += 1
          if (commit_count >= commit_limit): break
          
          commit_tokens = commit_msg_tokenizer(commit.msg)
          if (commit_tokens is None):
              continue
          
          # Check if changed files are python
          file_failed = False
          count_diff_lines = 0

          for mf in commit.modified_files:
              if (not mf.filename.endswith(".py")):
                  file_failed = True
                  break
              count_diff_lines += mf.added_lines + mf.deleted_lines
          
          if (file_failed):
              continue
          
          if (count_diff_lines > 50):
              continue
          
          # Create diff tokens
          diff_tokens = []
          for f in commit.modified_files:
            diff_tokens.append (['<file>'])
            diff_tokens.append(diff_tokenizer(f.diff))

          diff_whole_tokens = list(chain.from_iterable(diff_tokens))

          data.append([repo_name, commit.hash, json.dumps(commit_tokens), json.dumps(diff_whole_tokens)])
          commit_count += 1
        
      
      print(f"[DEBUG] {repo_name}: {len(data)} / {commit_count_traversed}")
      return pd.DataFrame(data, columns=["repo", "hash", "commit_messsage", "diff"])
    except Exception as e:
      print(e)
      print(repo_name)
      return pd.DataFrame([], columns=["repo", "hash", "commit_messsage", "diff"])


In [17]:
import time
start = time.time()
#df = parse_repo_commits("soimort/you-get")
df = parse_repo_commits("openthread/openthread")
end = time.time()
print(end - start)
df.head(10)

[DEBUG] openthread/openthread: 50 / 102
18.134236812591553


,repo,hash,commit_messsage,diff
0,openthread/openthread,b2c0c1a2af710225949f35ceb2af94504808796e,"[""use"", ""xa"", ""instead"", ""of"", ""dd"", ""in"", ""th...","[""<file>"", ""\n"", ""class"", ""test"", ""dnssd"", ""se..."
1,openthread/openthread,775c568286d22268def6ffd7da3723a938825259,"[""fix"", ""failing"", ""thread"", ""cert"", ""tests""]","[""<file>"", ""\n"", ""class"", ""open"", ""thread"", ""t..."
2,openthread/openthread,2d16fa25e4ec8ff1309400e1ae7548345a4335b9,"[""fix"", ""test"", ""publish"", ""meshcop"", ""service...","[""<file>"", ""\n"", ""class"", ""publish"", ""mesh"", ""..."
3,openthread/openthread,02d29cd5e343c3eb17840251b7feaa9345892895,"[""update"", ""meshcop"", ""service"", ""test"", ""case""]","[""<file>"", ""\n"", ""class"", ""publish"", ""mesh"", ""..."
4,openthread/openthread,cf4b71d8b90122f753486215557af46b25bc2185,"[""fix"", ""issue"", ""with"", ""not"", ""initialized"",...","[""<file>"", ""\n"", ""class"", ""open"", ""thread"", ""t..."
5,openthread/openthread,3dd184a0bc9577c2135f547fe16cd72454f6db26,"[""verify"", ""the"", ""number"", ""of"", ""published"",...","[""<file>"", ""\n"", ""class"", ""publish"", ""mesh"", ""..."
6,openthread/openthread,0e8b6604cb256cef828a0ebb37bdd031cdf0677d,"[""add"", ""set"", ""ccm"", ""state"", ""method"", ""in"",...","[""<file>"", ""\n"", ""class"", ""open"", ""thread"", ""t..."
7,openthread/openthread,e67f432284914aa6d2675a2df0507f44aa4a0e92,"[""fix"", ""for"", ""grl"", ""th"", ""leader"", ""leader""]","[""<file>"", ""\n"", ""class"", ""open"", ""thread"", ""t..."
8,openthread/openthread,130957d48981c1294c8fc90d51556134cf935d6f,"[""fix"", ""issue"", ""with"", ""is"", ""power"", ""down""...","[""<file>"", ""\n"", ""class"", ""open"", ""thread"", ""t..."
9,openthread/openthread,825227a71a6c70f8b046f5c66a91cde8abd14757,"[""fix"", ""mdns"", ""query""]","[""<file>"", ""\n"", ""eof"", ""\n"", ""print"", ""mdns"",..."


# 3. Create whole dataset via Multiprocessing

In [ ]:
import multiprocessing
import os

data = []

size = len(repodf['repo']) // 100

def f(repo):
  start = time.time()
  df = parse_repo_commits(repo)
  end = time.time()
  print(f"{repo} took {end-start} seconds")
  return df

for i in range (100):
    pool = multiprocessing.Pool()
    outputs = pool.map(f, repodf['repo'][size*i:min(size*(i+1), len(repodf['repo']))])
    pd.concat(outputs, ignore_index=True).to_pickle(f"{root}/repos/data{i}.pkl")

[DEBUG] google/gin-config: 50 / 100
google/gin-config took 4.605971574783325 seconds
[DEBUG] soimort/you-get: 50 / 80
soimort/you-get took 4.985018968582153 seconds
[DEBUG] graphql-python/graphene-sqlalchemy: 50 / 101
graphql-python/graphene-sqlalchemy took 5.735839366912842 seconds
[DEBUG] SectorLabs/django-postgres-extra: 50 / 115
SectorLabs/django-postgres-extra took 5.8941333293914795 seconds
[DEBUG] alpacahq/alpaca-trade-api-python: 50 / 117
alpacahq/alpaca-trade-api-python took 6.626330614089966 seconds
[DEBUG] ekzhu/datasketch: 46 / 155
ekzhu/datasketch took 9.250780582427979 seconds
[DEBUG] mopidy/mopidy-spotify: 50 / 105
mopidy/mopidy-spotify took 5.615523815155029 seconds
[DEBUG] ansible/ansible-lint: 50 / 107
ansible/ansible-lint took 10.703679084777832 seconds
[DEBUG] mozilla-iot/webthing-python: 20 / 76
mozilla-iot/webthing-python took 4.888516664505005 seconds
[DEBUG] Tivix/django-rest-auth: 50 / 85
Tivix/django-rest-auth took 6.132741212844849 seconds
[DEBUG] python/perf

In [ ]:
df = pd.read_pickle(f"{root}/data.pkl")
df.head(3)

,repo,hash,commit_messsage,diff
0,soimort/you-get,5a12118c836b0b9a7a2f8bdbed25be9aa2ca7831,"[""add"", ""param"", ""postfix"", ""to"", ""postfix"", ""...","[""<file>"", ""\n"", ""output"", ""filename"", ""= "", ""..."
1,soimort/you-get,3f47a215bf5f73e73cd7aa607b340a7b68b9bd3d,"[""fix"", ""the"", ""name"", ""m"", ""u"", ""is"", ""not"", ...","[""<file>"", ""\n"", ""cookies"", ""= "", ""none"", ""\n""..."
2,soimort/you-get,424b555500e65da0533d369a80562bb93907ec9a,"[""fix"", ""zhihu"", ""extractor"", ""comment"", ""ld""]","[""<file>"", ""\n"", ""def"", ""zhihu"", ""download"", ""..."
